In [1]:
import tensorflow as tf
import os
import sys
import numpy as np
import math

tf.logging.set_verbosity(tf.logging.INFO)



In [2]:
## 下面高参数 10K 次 train: 99.7%, Valid: 92.7%, Test: 93.2%
def get_default_params():
    return tf.contrib.training.HParams(
        # num_embedding_size = 16,
        num_embedding_size = 32,
        # 一个句子取前50个分词
        # num_timesteps = 50,
        num_timesteps = 600,
        # num_filters = 128,
        num_filters = 256,
        num_kernel_size = 3,
        num_fc_nodes = 32,
        # num_fc_nodes = 64,
        batch_size = 100,
        learning_rate = 0.001,
        num_word_threshold = 10,
    )
hps = get_default_params()

train_file = '.\deep_learn\sohu_seg_train_file.txt'
test_file = '.\deep_learn\sohu_seg_test_file.txt'

vocab_file =  '.\deep_learn\sohu_vocab.txt'
category_file = '.\deep_learn\sohu_category.txt'
output_file = '.\deep_learn\sohu_run_text_run'

if not os.path.exists(output_file):
    os.mkdir(output_file)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
# api sentence_to_id的实现
class Vocab:
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
    
    def _read_dict(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id)
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)
    
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
    
    def sentence_to_id(self, sentence):
        # 分词后的句子用空格来隔开每一个词语,这里每一句话里的分词用id来表示
        word_ids = [self.word_to_id(cur_word) for cur_word in sentence.split()]
        return word_ids

class CategoryDict:
    def __init__(self, filename):
        self._category_to_id = {}
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx
            
    def size(self):
        return len(self._category_to_id)
    
    def category_to_id(self, category):
        if not category in self._category_to_id:
            print(self._category_to_id)
            raise Exception("{} is not in our category".format(category))
            
        return self._category_to_id[category]
          
vocab = Vocab(vocab_file, hps.num_word_threshold)
vocab_size = vocab.size()

tf.logging.info('vocab_size: {}'.format(vocab_size))
category_vocab = CategoryDict(category_file)
num_classes = category_vocab.size()
tf.logging.info('category_size: {}'.format(num_classes))
test_str = '女人'
tf.logging.info('id:{}'.format(category_vocab.category_to_id(test_str)))

INFO:tensorflow:vocab_size: 85428
INFO:tensorflow:category_size: 12
INFO:tensorflow:id:11


In [4]:
# 
class TextDataSet:
    def __init__(self, filename, vocab, category_vocab, num_timesteps):
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        
        self._inputs = []
        self._outputs = []
        
        self._indicator = 0
        self._parse_file(filename)
        
    def _parse_file(self, filename):
        tf.logging.info('Loading data from {}'.format(filename))
        lines = 0
        import re
        if re.findall('train', filename):
            with open(filename, 'r') as f:
                lines = f.readlines()
        elif re.findall('test', filename):
            with open(filename, 'r', encoding='utf-8-sig') as f:
                lines = f.readlines()  
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            id_label = self._category_vocab.category_to_id(label)
            id_words = self._vocab.sentence_to_id(content)
            
            id_words = id_words[0: self._num_timesteps]
            padding_num = self._num_timesteps - len(id_words)
            id_words = id_words + [
                self._vocab.unk for i in range(padding_num)]
            self._inputs.append(id_words)
            self._outputs.append(id_label)

        
        self._inputs = np.asarray(self._inputs, dtype=np.int32)
        self._outputs = np.asarray(self._outputs, dtype=np.int32)
        self._random_shuffle()
        
    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
        self._inputs = self._inputs[p]
        self._outputs = self._outputs[p]
    
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        if end_indicator > len(self._inputs):
            raise Exception("batch size: {} is too large".format(batch_size))
        
        batch_inputs = self._inputs[self._indicator: end_indicator]
        batch_output = self._outputs[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_inputs, batch_output

train_dataset = TextDataSet(train_file, vocab, category_vocab, hps.num_timesteps)
test_dataset = TextDataSet(test_file, vocab, category_vocab, hps.num_timesteps)

print(train_dataset.next_batch(2))
print(test_dataset.next_batch(2))
# 因为next_batch == 2 ,输出为
'''(array([[ 3879,  3048,     0,  1302,  6449,  9225, 32020, 75465,    26,
          938,    25, 42449,  1743,  5589,  1071, 28916,   875,     7,
         1071,  9306, 13826, 20919,   190,     1,  5614,  7024,   359,
        68957,   559,  9255, 11563,   172,  2431,   627,  7521,    51,
         5280, 15051,    50,    94,     0,     0,    65,   938, 42449,
           90,  2603,  2234,     8,  3420],
       [  192,   116,   310, 21315,  4187,  7290,  2223,  5918,    11,
        33789,    12,   922,    13,  6187,  1061,     7, 37987,  4187,
            2,   192,   116,   310,   880, 25294,     1,   104,   675,
         9024,     2,  1643,    10,    11,  8970,    12,     3,   968,
            1,     7,    11,  6890,   639,    13,   310,    42,     9,
            4,     4,    18,  3163,  3163]]), array([ 9, 10]))
'''
# 只选择句子前50个分词, 输出为2*50的矩阵,第一个为句子分词的索引id, 第二个为分类的索引id


INFO:tensorflow:Loading data from .\deep_learn\sohu_seg_train_file.txt
INFO:tensorflow:Loading data from .\deep_learn\sohu_seg_test_file.txt
(array([[    7,    27,    14, ...,     2,   165,     1],
       [21154,   373,    13, ...,   363,    73,   572]]), array([11,  4]))
(array([[21562,  9878,    77, ...,  9728,     2,    31],
       [  725,  7560,   146, ...,  1967, 29053,     0]]), array([10,  6]))


'(array([[ 3879,  3048,     0,  1302,  6449,  9225, 32020, 75465,    26,\n          938,    25, 42449,  1743,  5589,  1071, 28916,   875,     7,\n         1071,  9306, 13826, 20919,   190,     1,  5614,  7024,   359,\n        68957,   559,  9255, 11563,   172,  2431,   627,  7521,    51,\n         5280, 15051,    50,    94,     0,     0,    65,   938, 42449,\n           90,  2603,  2234,     8,  3420],\n       [  192,   116,   310, 21315,  4187,  7290,  2223,  5918,    11,\n        33789,    12,   922,    13,  6187,  1061,     7, 37987,  4187,\n            2,   192,   116,   310,   880, 25294,     1,   104,   675,\n         9024,     2,  1643,    10,    11,  8970,    12,     3,   968,\n            1,     7,    11,  6890,   639,    13,   310,    42,     9,\n            4,     4,    18,  3163,  3163]]), array([ 9, 10]))\n'

In [5]:
def create_model(hps, vocab_size, num_classes):
    # 取一个句子的前50个分词, num_classes为固定的50个分词
    num_timesteps = hps.num_timesteps
    # 训练批次大小
    batch_size = hps.batch_size
    # 输入为[批次的大小,50]
    inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    # 输出为[批次的大小,]
    outputs = tf.placeholder(tf.int32, (batch_size, ))
    # dropout的使用
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    # 保存训练到哪一步
    global_step = tf.Variable(
        tf.zeros([], tf.int64), name='global_step', trainable=False)
    # 随机化embedding 编码
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope(
        'embedding', initializer=embedding_initializer):
        embeddings = tf.get_variable(
            'embedding',
            [vocab_size, hps.num_embedding_size],
            tf.float32)
        # 把输入的分词中的id -> embedding编码形式
        # ex [1, 10, 7] -> [embeddings[1], embeddings[10], embeddings[7]]
        embed_inputs = tf.nn.embedding_lookup(embeddings, inputs)
    # 卷积实现
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_filters) / 3.0
    cnn_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('cnn', initializer= cnn_init):
        # embed_inputs : [batch_size, timesteps, embed_size]
        # conv1d : [batch_size, timesteps, num_filters]
        conv1d = tf.layers.conv1d(embed_inputs,
                                  hps.num_filters,
                                  hps.num_kernel_size,
                                  activation=tf.nn.relu,
                                  )
        global_maxpooling = tf.reduce_max(conv1d, axis=[1])
    
     
    """
    # 网络initializer的一种方法
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_lstm_nodes[-1]) / 3.0
    lstm_init = tf.random_uniform_initializer(-scale, scale)
    # 构建lstm
    with tf.variable_scope('lstm_nn', initializer=lstm_init):
        cells = []
        for i in range(hps.num_lstm_laysers):
            # 循环初始化lstm
            cell = tf.contrib.rnn.BasicLSTMCell(
                hps.num_lstm_nodes[i],
                state_is_tuple = True
            )
            # 使用dropout方法
            cell = tf.contrib.rnn.DropoutWrapper(
                cell,
                output_keep_prob = keep_prob
            )
            cells.append(cell)
        # 合并两个cell
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        # 初始化cell内的值
        initial_state = cell.zero_state(batch_size, tf.float32)
        # run_outputs: [batch_size, num_timesteps, lstm_outpus[-1]]
        run_outpus, _ = tf.nn.dynamic_rnn(
            cell, embed_inputs, initial_state=initial_state
        )
        print(run_outpus)
        last = run_outpus[:, -1, :]
    """
    fc_init = tf.uniform_unit_scaling_initializer(factor=1.0)
    # cnn连接到全连接层
    with tf.variable_scope('fc', initializer=fc_init):
        fc1 = tf.layers.dense(global_maxpooling,
                              hps.num_fc_nodes,
                              activation=tf.nn.relu,
                              name='fc1')
        # 使用dropout方法
        fc1_dropout = tf.contrib.layers.dropout(fc1, keep_prob)
        logits = tf.layers.dense(fc1_dropout,
                                 num_classes,
                                 name='fc2')
    # 计算损失函数
    with tf.name_scope('metrics'):
        sofmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels= outputs
        )
        loss = tf.reduce_mean(sofmax_loss)
        y_pred = tf.arg_max(tf.nn.softmax(logits=logits),
                            1,
                            output_type= tf.int32)
        correct_pred = tf.equal(outputs, y_pred)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    # 构建train_op
    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(hps.learning_rate).minimize(loss,
                                                                      global_step=global_step)
        """
        tvars = tf.trainable_variables()
        for var in tvars:
            tf.logging.info('variable name: {}'.format(var.name))
        # 限制训练时的梯度大小,使得不会出现梯度爆炸
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(loss, tvars), hps.clip_lstm_grads
        )
        # 梯度应用到变量中去
        optimizer = tf.train.AdamOptimizer(hps.learning_rate)
        train_op = optimizer.apply_gradients(
            zip(grads, tvars), global_step= global_step
        )
        """
    return ((inputs, outputs, keep_prob),
            (loss, accuracy),
            (train_op, global_step))

placeholders, metrics, others = create_model(
    hps, vocab_size, num_classes
)
inputs, outputs, keep_prod = placeholders
loss, accuracy = metrics
train_op, global_step = others

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.math.argmax` instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [6]:
# train:
init_op = tf.global_variables_initializer()
train_keep_prob_value = 0.8
test_keep_prob_value = 1.0

test_steps = 100
num_train_steps = 10000

# Train : 100%
# Test : 95.3%

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        batch_inputs, batch_labels = train_dataset.next_batch(
            hps.batch_size
        )
        outputs_val = sess.run([loss, accuracy, train_op, global_step],
                               feed_dict = {
                                inputs: batch_inputs,
                                outputs: batch_labels,
                                   keep_prod: train_keep_prob_value,
                               })
        loss_val, accuracy_val, _, global_step_val = outputs_val
        if global_step_val % 100 == 0:
            tf.logging.info("Train Step: {}, loss: {}, accuracy: {}".format(global_step_val, loss_val, accuracy_val))

        if global_step_val % 1000 == 0:
            all_test_acc_cal = []
            for j in range(test_steps):
                test_inputs, test_labels = test_dataset.next_batch(hps.batch_size)
                test_val = sess.run([loss, accuracy, train_op, global_step],
                                    feed_dict= {
                                        inputs: test_inputs,
                                        outputs: test_labels,
                                        keep_prod: test_keep_prob_value,
                                    })
                test_loss_val, test_accuarcy_val, _, test_step_val = test_val
                all_test_acc_cal.append(test_accuarcy_val)
            test_acc = np.mean(all_test_acc_cal)
            tf.logging.info("------Test Step: {}, loss: {}, accuracy: {}".format(global_step_val, test_loss_val, test_acc))

INFO:tensorflow:Train Step: 100, loss: 2.1734323501586914, accuracy: 0.1899999976158142
INFO:tensorflow:Train Step: 200, loss: 1.6766830682754517, accuracy: 0.46000000834465027
INFO:tensorflow:Train Step: 300, loss: 1.199046015739441, accuracy: 0.6499999761581421
INFO:tensorflow:Train Step: 400, loss: 0.9451839923858643, accuracy: 0.7099999785423279
INFO:tensorflow:Train Step: 500, loss: 0.9733993411064148, accuracy: 0.6899999976158142
INFO:tensorflow:Train Step: 600, loss: 1.0734984874725342, accuracy: 0.6700000166893005
INFO:tensorflow:Train Step: 700, loss: 0.8732958436012268, accuracy: 0.75
INFO:tensorflow:Train Step: 800, loss: 0.5862252116203308, accuracy: 0.8199999928474426
INFO:tensorflow:Train Step: 900, loss: 0.6773641109466553, accuracy: 0.7900000214576721
INFO:tensorflow:Train Step: 1000, loss: 0.6384667754173279, accuracy: 0.800000011920929
INFO:tensorflow:------Test Step: 1000, loss: 0.4643302261829376, accuracy: 0.8248999714851379
INFO:tensorflow:Train Step: 1200, loss: 